In [401]:
import os
import json
cwd = os.getcwd();

In [402]:
mode="toOBJ"
modelName = "ionic_capital"
outputOBJ = ""
vertexes = {}
uniqueVertexes = {}
normals = {}
uniqueNormals= {}
uv = {}
uniqueUV = {}
faces = {}
#(face,localIndex),remappedIndex
faceVertexMap = {}
faceUVMap = {}
faceNormalMap = {}
if(mode=="toOBJ"):
    filename="test_inputs/"+modelName+".smeg"
else:
    filename="test_inputs/"+modelName+".obj"
#fileName = input("Input File: ");


In [403]:
def smeg_to_obj(modelFile):
    global modelName
    global outputOBJ
    try: 
        inputFile = open(modelFile, "r");
        modelData = json.load(inputFile);
        outputOBJ = outputOBJ + "mtllib " + modelName + ".mtl\n"
        outputOBJ = outputOBJ + "o " + modelName + "\n"
        j = 0
        for f in modelData['faces']:
            i=0
            for v in f['vertices']:
                vertexes.update({(v[0],v[1],v[2],j): (i,i)})
                uniqueVertexes.update({(v[0],v[1],v[2]):""})
    
                normals.update({(v[3],v[4],v[5],j,i): (i,i)})
                uniqueNormals.update({(v[3],v[4],v[5]): ""})
    
                uv.update({(v[6],v[7],j): (i,i)})
                uniqueUV.update({(v[6],v[7]): ""})
                i+=1
            for t in f['triangles']:
                faces.update({(t[0],t[1],t[2],j):(t[0],t[1],t[2])})
            j+=1

        
        #index uniqueVertexes and put them in the output file
        i = 0
        for v in uniqueVertexes:
            uniqueVertexes.update({v:i})
            outputOBJ += "v "+ str(v[0]) + " " + str(v[1]) + " "+ str(v[2]) + "\n"
            i+=1
        
        i = 0
        for u in uniqueUV:
            uniqueUV.update({u:i})
            #outputOBJ += "vn "+ str(u[0]) + " " + str(u[1]) + "\n"
            i+=1
          
        i = 0
        for n in uniqueNormals:
            uniqueNormals.update({n:i})
            outputOBJ += "vt "+ str(n[0]) + " " + str(n[1]) + " "+ str(n[2]) + "\n"
            i+=1
        
        
        #matches face vertices to the unique mapping of vertices
        #print("vertexes")#v
        for v in vertexes:
            vertexes.update({v: (vertexes[v][0],uniqueVertexes[(v[0],v[1],v[2])])})
            faceVertexMap.update({(v[3],vertexes[v][0]):vertexes[v][1]})
            #print(v," | ", vertexes[v])
        
        #print("\nuv")#v
        for u in uv:
            uv.update({u: (uv[u][0],uniqueUV[(u[0],u[1])])})
            faceUVMap.update({(u[2],uv[u][0]):uv[u][1]})
            #print(u," | ", uv[u])      
        #print("\nnormals")#v
        for n in normals:
            normals.update({n: (normals[n][0],uniqueNormals[(n[0],n[1],n[2])])})
            faceNormalMap.update({(n[3],normals[n][0]):normals[n][1]})
            #print(n," | ", normals[n])    
        outputOBJ += "usemtl None\ns off\n"
            
        #print(str(normals) + "\n") #vn
        #print(str(uv) + "\n") #vt
        #print(str(faces) + "\n") #f
        #print(faceNormalMap)
        for f in faces:   
            #x = str(faceVertexMap[(f[3],f[0])]+1) + "/" + str(faceUVMap[(f[3],f[0])]+1) + "/" + str(faceNormalMap[(f[3],f[0])]+1)
            #y = str(faceVertexMap[(f[3],f[1])]+1) + "/" + str(faceUVMap[(f[3],f[1])]+1) + "/" + str(faceNormalMap[(f[3],f[1])]+1)
            #z = str(faceVertexMap[(f[3],f[2])]+1) + "/" + str(faceUVMap[(f[3],f[2])]+1) + "/" + str(faceNormalMap[(f[3],f[2])]+1)
            x = str(faceVertexMap[(f[3],f[0])]+1) + "//" + str(faceNormalMap[(f[3],f[0])]+1)
            y = str(faceVertexMap[(f[3],f[1])]+1) + "//" + str(faceNormalMap[(f[3],f[1])]+1)
            z = str(faceVertexMap[(f[3],f[2])]+1) + "//" + str(faceNormalMap[(f[3],f[2])]+1)
            outputOBJ += "f " + x + " " + y + " " + z +"\n"
        output = open("test_outputs/"+modelName+".obj","w")
        output.write(outputOBJ)
        output.close()
        
        
        inputFile.close()
    except IOError:
        print("File not accessible");  

In [404]:

def obj_to_smeg(modelFile):
    try:  
        inputFile = open(modelFile, "r");
        inputLines = inputFile.readlines();
        i=1
        for line in inputLines:
            tokens = line.split(' ')
            if(tokens[0]=='#'):
                print("Comment")
            elif(tokens[0]=='mtllib'):
                print("Materials")
            elif(tokens[0]=='o'):
                print("Object Name")
            elif(tokens[0]=='v'):
                print("Vertex")
                vertexes.update({i:[tokens[1],tokens[2],tokens[3].replace('\n','')]})
                i += 1
            elif(tokens[0]=='vt'):
                print("UV map vertex")
            elif(tokens[0]=='usemtl'):
                print("Material to use")
            elif(tokens[0]=='s'):
                print("Smooth Shading")
            elif(tokens[0]=='f'):
                print("Face")
        for x in vertexes:
            print(x)
        inputFile.close()
    except IOError:
        print("File not accessible");  

In [405]:

if(mode=="toOBJ"):
    smeg_to_obj(filename)
else:
    obj_to_smeg(filename)